# [문제4]
- law_1.docx, law_2.docx(이미지포함) 둘다.  
- pinecone store 저장
    - index name 임의
- RetrievalQA 구현 
    - prompt: rlm/rag-prompt
    - 질문: 전세사기

In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document

load_dotenv()


loader1 = Docx2txtLoader("law_1.docx")
loader2 = Docx2txtLoader("law_2.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
)

data_list = []
data_list1 = loader1.load_and_split(text_splitter=text_splitter)
data_list2 = loader2.load_and_split(text_splitter=text_splitter)

formula_1 = """
제3조(국세의 안분 방법 및 신청 등) 제1항 제1호 가목

가. 다음의 계산식에 따라 계산한 금액:

A × B ÷ C

A: 전세사기피해주택의 임대인이 세납한 고지 또는 신고 건별 국세 금액  
B: 전세사기피해주택의 가격  
C: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액
"""

doc_formula_1 = Document(page_content=formula_1, metadata={"source": "수식 이미지 1"})

formula_2 = """
제4조(지방세의 안분 방법 및 신청 등) 제1항 제2호

2. 제1호 외의 지방세의 경우: 고지 또는 신고 건별로 각각 다음의 계산식에 따라 계산한 금액:

A × B ÷ C

A: 전세사기피해주택의 임대인이 세납한 고지 또는 신고 건별 지방세 금액  
B: 전세사기피해주택의 가격  
C: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액
"""

doc_formula_2 = Document(page_content=formula_2, metadata={"source": "수식 이미지 2"})

data_list.extend(data_list1)
data_list.extend(data_list2)
data_list.extend([doc_formula_1, doc_formula_2])
# 어펜드는 [[]] 데이터리스트1(리스트)를 통채로 넣는다면 extend는 리스트의 요소를 뽑아서 더함. 

## 임베딩
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

# 파인콘에 저장
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

# database = PineconeVectorStore.from_documents(
#     documents=data_list,
#     embedding=embedding,
#     index_name='law1and2-quiz4'
# )

database = PineconeVectorStore(
    embedding=embedding,
    index_name='law1and2-quiz4'
)

llm = ChatOpenAI()

prompt = hub.pull("rlm/rag-prompt")

## RetrievalQA 구현
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

chain_lcel = (
    {
        "context" : database.as_retriever() | format_docs,
        "question" : RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain_lcel.invoke("제 3조에 따라 안분하여 징수하는 전세사기피해주택 임대인의 국세 계산식을 알려주세요.")

'전세사기피해주택 임대인의 국세 계산식은 A × B ÷ C로 계산됩니다. 여기서 A는 세납한 고지 또는 신고 건별 국세 금액, B는 전세사기피해주택의 가격, C는 임대인이 보유한 모든 주택의 가격 합계액을 나타냅니다.'